### Modelo de Regressión Lineal con SparkML y Python

La regresión lineal es de aprendizaje supervisado y se utiliza en análisis de tendencias, análisis de series de tiempo, riesgo en la banca y muchos más. Para nuestro caso, lo usaremos para predecir el costo de un viaje en taxi.

En una regresión lineal, una relación entre una variable dependiente y un conjunto de datos es lineal. Esto básicamente significa que si hay datos de una tendencia específica, se puede predecir una tendencia futura.

In [1]:
# medir tiempos
%load_ext autotime

time: 0 ns (started: 2021-06-14 12:53:52 -05:00)


#### Iniciando un entorno de trabajo con Spark

In [2]:
import findspark
#findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")  #para linux
findspark.init()                                               #para windows

time: 16 ms (started: 2021-06-14 12:53:52 -05:00)


In [3]:
from pyspark import SparkConf, SparkContext
# Variable de configuración
conf = SparkConf().setMaster("local[*]").setAppName("LinearRegression")  \
    .set("spark.driver.maxResultSize","0") \
    .set("spark.driver.memory", "8g") \
    .set("spark.cores.max", "8")
    #.set("spark.executor.memory", "1g") \
    
# Iniciamos un contexto spark (solo se ejecuta uno. Para ejecutar otra vez , reiniciar el kernel)
sc = SparkContext(conf = conf)
sc

<SparkContext master=local[*] appName=LinearRegression>

time: 4.01 s (started: 2021-06-14 12:53:53 -05:00)


#### Cargando el conjunto de datos

In [4]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
# le pasamos el contexto anterior
sqlContext = SQLContext(sc)
import os
# path="file:"+os.getcwd()+"/Output"           # para linux
path=os.getcwd()+"/Output"                     # para windows
dfspark = sqlContext.read.format('csv').option("header","true").option("inferSchema","true").load(path+'/*.csv')

time: 8.17 s (started: 2021-06-14 12:53:57 -05:00)


In [5]:
dfspark.printSchema()

root
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- dif_latitude: double (nullable = true)
 |-- dif_longitude: double (nullable = true)
 |-- distancia: double (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- anio: integer (nullable = true)

time: 16 ms (started: 2021-06-14 12:54:05 -05:00)


La característica pickup_datetime ya se usó para crear nuevos datos, por lo que procederemos a eliminarla antes de construir el modelo de regresión.

In [6]:
dfspark = dfspark.drop('pickup_datetime')

time: 0 ns (started: 2021-06-14 12:54:05 -05:00)


Preparando el conjunto de datos para el aprensizaje automático. Sólo se usará dos columnas, la etiqueta (fare_amount) que es valor a predecir, y otra columna de características. Para ello se importa VectoAssembler que transformará los datos en un vector de carácterísticas.

In [7]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','distancia', 'dia_semana', 'hora', 'mes', 'anio','passenger_count'], outputCol = 'features')
dfspark = vectorAssembler.transform(dfspark)
dfspark = dfspark.select(['features', 'fare_amount'])
dfspark.show(3)

+--------------------+-----------+
|            features|fare_amount|
+--------------------+-----------+
|[-73.961407,40.71...|       13.0|
|[-74.011287,40.70...|       45.0|
|[-73.991907,40.73...|       16.9|
+--------------------+-----------+
only showing top 3 rows

time: 578 ms (started: 2021-06-14 12:54:05 -05:00)


División del conjunto de datos usando randomsplit, 80% para entrenamiento y 20% para prueba

In [8]:
splits = dfspark.randomSplit([0.8, 0.2],seed=0)
train_df = splits[0]
test_df = splits[1]

time: 32 ms (started: 2021-06-14 12:54:06 -05:00)


#### Linear Regression

In [9]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='fare_amount', maxIter=30, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(dfspark)
print("Coeficientes: " + str(lr_model.coefficients))
print("Intercepto: " + str(lr_model.intercept))

Coeficientes: [0.0,0.0,0.0,0.0,0.00038516866757571153,0.0,-0.012793274508220114,0.07589766478781158,0.5461330399276331,0.03932990644267434]
Intercepto: -1087.7769061927456
time: 6.8 s (started: 2021-06-14 12:54:06 -05:00)


In [10]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 9.319407
r2: 0.015916
time: 0 ns (started: 2021-06-14 12:54:21 -05:00)


In [11]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","fare_amount","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="fare_amount",metricName="r2")
print("R al cuadrado (R2) en datos de prueba = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|fare_amount|            features|
+------------------+-----------+--------------------+
|11.282150381536894|       16.9|(10,[5,6,7,8,9],[...|
|10.811915006397157|       12.9|(10,[5,6,7,8,9],[...|
|10.177091027173446|       45.0|(10,[5,6,7,8,9],[...|
|10.723224067101228|        5.7|(10,[5,6,7,8,9],[...|
|10.723224067101228|       13.3|(10,[5,6,7,8,9],[...|
+------------------+-----------+--------------------+
only showing top 5 rows

R al cuadrado (R2) en datos de prueba = 0.015516
time: 7.58 s (started: 2021-06-14 12:54:39 -05:00)


In [12]:
test_result = lr_model.evaluate(test_df)
print("Error cuadrático medio (RMSE) en los datos de prueba = %g" % test_result.rootMeanSquaredError)

Error cuadrático medio (RMSE) en los datos de prueba = 9.33447
time: 3.42 s (started: 2021-06-14 12:59:27 -05:00)
